# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import csv

from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.widgets import RunDetails

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import normal, choice
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-capstone-experiment")

run = exp.start_logging()

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "capstone-compute"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()
experiment_name = 'udacity-capstone-penguins'

experiment=Experiment(ws, experiment_name)

experiment

Name,Workspace,Report Page,Docs Page
udacity-capstone-penguins,udacity_ml_azure_capstone,Link to Azure Machine Learning studio,Link to Documentation


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
packages = CondaDependencies.create(
    pip_packages=['azureml-defaults','scikit-learn']
)

# Specify a Policy: check the job every 2 iterations and if the primary metric falls outside the top 50% range, 
# Azure ML should terminate the job
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

early_termination_policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)

#TODO: Create the different params that you will be using during training

param_sampling = RandomParameterSampling({
    '--C': choice(0.1, 0.2, 0.3),
    '--max_iter': choice(10, 20, 30)
})

# Setup environment for your training run

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config

src = ScriptRunConfig(source_directory="./",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)






In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_run_config)

# Show run details
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d
Web View: https://ml.azure.com/runs/HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d?wsid=/subscriptions/a022d83d-6229-4b5e-b039-b680692436b5/resourcegroups/aspatton01-rg/workspaces/udacity_ml_azure_capstone&tid=41b3409a-40cf-45e8-8201-ddfa1984ab3d

Streaming azureml-logs/hyperdrive.txt

[2024-03-20T18:38:01.601970][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-03-20T18:38:02.2603411Z][SCHEDULER][INFO]Scheduling job, id='HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_1' 
[2024-03-20T18:38:02.2265898Z][SCHEDULER][INFO]Scheduling job, id='HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_0' 
[2024-03-20T18:38:02.3812686Z][SCHEDULER][INFO]Scheduling job, id='HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_2' 
[2024-03-20T18:38:02.4720932Z][SCHEDULER][INFO]Scheduling job, id='HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_3' 
[2024-03-20T18:38:02.442419][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.

hd_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hd_best_model = hd_best_run.register_model(model_name="hd_best_model" , model_path="./")

best_run_metrics = hd_best_run.get_metrics()
parameter_values = hd_best_run.get_details()
best_run_filenames = hd_best_run.get_file_names()


print("Best Run Id: ", hd_best_run.id)
print("Best run metrics: ", best_run_metrics)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Best run details ", parameter_values)
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

Best Run Id:  HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_1
Best run metrics:  {'Regularization Strength:': 0.3, 'Max iterations:': 20, 'Accuracy': 0.7536231884057971}
Accuracy:  0.7536231884057971
Best run details  {'runId': 'HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_1', 'target': 'capstone-compute', 'status': 'Completed', 'startTimeUtc': '2024-03-20T18:38:12.228295Z', 'endTimeUtc': '2024-03-20T18:39:15.977771Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcdsi', '_azureml.ClusterName': 'capstone-compute', 'ContentSnapshotId': '1d79c9c4-8982-493e-a131-fac99642f406', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [{'dataset': {'id': 'b8d5f66a-9d99-4f9b-b6b8-62f8873e4bf2'}, 'consumptionDetails': {'type': 'Reference'}}], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.3', '--max_iter', '20'], 'sourceDirectoryDataSto

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
hd_best_run.get_details()

{'runId': 'HD_8ce5ea1a-0a6b-4f56-9230-12e6f697dd7d_1',
 'target': 'capstone-compute',
 'status': 'Completed',
 'startTimeUtc': '2024-03-20T18:38:12.228295Z',
 'endTimeUtc': '2024-03-20T18:39:15.977771Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcdsi',
  '_azureml.ClusterName': 'capstone-compute',
  'ContentSnapshotId': '1d79c9c4-8982-493e-a131-fac99642f406',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'b8d5f66a-9d99-4f9b-b6b8-62f8873e4bf2'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '0.3', '--max_iter', '20'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'capstone-compute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': No

In [11]:
print(hd_best_run.get_file_names())

model = hd_best_run.register_model(model_name = "CapstoneBest", model_path=".")


hd_best_run.download_file("/outputs/model.joblib","linreg_best_model.joblib")

['logs/azureml/dataprep/0/rslex.log.2024-03-20-18', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

